**Image Processing Project Image Processing and Computer Vision Techniques used in Autonomous Vehicles**

**Canny Edge Detection**

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [2]:
import cv2 
image = cv2.imread('car.jpg')
cv2.imshow('Car', image)
cv2.waitKey()
cv2.destroyAllWindows()

**Converting to grayscale**

In [3]:
image.shape

(554, 1024, 3)

In [4]:
img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('Car in Grayscale', img)
cv2.waitKey()
cv2.destroyAllWindows()

**Canny Edge Detection**

In [ ]:
def Canny_detector(img, weak_th = None, strong_th = None):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (5, 5), 1.4)
    gx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0, 3)
    gy = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1, 3)
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees = True)
    mag_max = np.max(mag)
    if not weak_th:weak_th = mag_max * 0.1
    if not strong_th:strong_th = mag_max * 0.5
    
    height, width = img.shape
    
    for i_x in range(width):
        for i_y in range(height):
            grad_ang = ang[i_y, i_x]
            grad_ang = abs(grad_ang-180) if abs(grad_ang)>180 else abs(grad_ang)
            
            if grad_ang<= 22.5:
                neighb_1_x, neighb_1_y = i_x-1, i_y
                neighb_2_x, neighb_2_y = i_x + 1, i_y
            
            elif grad_ang>22.5 and grad_ang<=(22.5 + 45):
                neighb_1_x, neighb_1_y = i_x-1, i_y-1
                neighb_2_x, neighb_2_y = i_x + 1, i_y + 1
            
            elif grad_ang>(22.5 + 45) and grad_ang<=(22.5 + 90):
                neighb_1_x, neighb_1_y = i_x, i_y-1
                neighb_2_x, neighb_2_y = i_x, i_y + 1
                
            elif grad_ang>(22.5 + 90) and grad_ang<=(22.5 + 135):
                neighb_1_x, neighb_1_y = i_x-1, i_y + 1
                neighb_2_x, neighb_2_y = i_x + 1, i_y-1
                
            elif grad_ang>(22.5 + 135) and grad_ang<=(22.5 + 180):
                neighb_1_x, neighb_1_y = i_x-1, i_y
                neighb_2_x, neighb_2_y = i_x + 1, i_y
            
            if width>neighb_1_x>= 0 and height>neighb_1_y>= 0:
                if mag[i_y, i_x]<mag[neighb_1_y, neighb_1_x]:
                    mag[i_y, i_x]= 0
                    continue
            
            if width>neighb_2_x>= 0 and height>neighb_2_y>= 0:
                if mag[i_y, i_x]<mag[neighb_2_y, neighb_2_x]:
                    mag[i_y, i_x]= 0
            
    weak_ids = np.zeros_like(img)
    strong_ids = np.zeros_like(img)
    ids = np.zeros_like(img)
    
    for i_x in range(width):
        for i_y in range(height):
            grad_mag = mag[i_y, i_x]
            
            if grad_mag<weak_th:
                mag[i_y, i_x]= 0
                
            elif strong_th>grad_mag>= weak_th:
                ids[i_y, i_x]= 1
                
            else:
                ids[i_y, i_x]= 2
                
    return mag

canny_img = Canny_detector(image)
cv2.imshow('Original Image', image)
cv2.imshow('Canny Edge Detection', canny_img)
cv2.waitKey()
cv2.destroyAllWindows()
                
            
        
    
    
                    
            
            
        

**Canny Edge Detection with Webcam Feed**

In [ ]:
import cv2
import numpy as np
def Webcam_Canny(image_color):
            
    threshold1 = 30
    threshold2 = 100
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(image_gray, threshold1, threshold2)  
    return canny

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, vid = cap.read()
    cv2.imshow('Live Edge Detection', Webcam_Canny(vid))
    #cv2.imshow('Webcam Video', vid)
    if cv2.waitKey(1) == 13: #Enter Key to exit
        break
        
cap.release()
cv2.destroyAllWindows()

# Lane Detection using Hough Transform

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib.image as mpimg

In [ ]:
image_c = mpimg.imread('lane_detect.jpg')
plt.imshow(image_c)  
image_c.shape

In [ ]:
image_g = cv2.cvtColor(image_c, cv2.COLOR_RGB2GRAY)
plt.imshow(image_g, cmap = 'gray')  
image_g.shape

In [ ]:
image_blurred = cv2.GaussianBlur(image_g, (7, 7), 0)
plt.imshow(image_blurred)  

In [ ]:
threshold_low = 10
threshold_high = 200
image_canny = cv2.Canny(image_blurred, threshold_low, threshold_high)
plt.imshow(image_canny, cmap = 'gray')  

**Region of Interest**

In [ ]:
vertices = np.array([[(20,460),(340, 300), (460, 300), (740,460)]], dtype=np.int32)  
#vertices = np.array([[(200,550),(250, 180), (550, 180), (700,550)]], dtype=np.int32)  
mask = np.zeros_like(image_g)   
cv2.fillPoly(mask, vertices, 255)
masked_image = cv2.bitwise_and(image_g, mask)
plt.figure()
plt.imshow(masked_image)

In [ ]:
masked_image = cv2.bitwise_and(image_canny, mask)
plt.figure()
plt.imshow(masked_image)

In [ ]:
rho = 2             
theta = np.pi/180  
threshold = 40     
min_line_len = 100 
max_line_gap = 50  
lines = cv2.HoughLinesP(masked_image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    

line_image = np.zeros((masked_image.shape[0], masked_image.shape[1], 3), dtype=np.uint8)
    
for line in lines:
    for x1,y1,x2,y2 in line:      
        cv2.line(line_image, (x1, y1), (x2, y2), [255, 0, 0], 20)
lines 

In [ ]:
α = 1
β = 1
γ = 0    
Image_with_lines = cv2.addWeighted(image_c, α, line_image, β, γ)

In [ ]:
plt.figure()
plt.imshow(Image_with_lines)
cv2.imshow('Lane Detected', Image_with_lines)
cv2.waitKey()
cv2.destroyAllWindows()